In [21]:
# Load Python traces

import pandas as pd
import json
import re
from pprint import pp
from tree_sitter_languages import get_parser
import os

parser = get_parser("python")

processed_count = 0


projects_path = "/home/XXX/Traces/Python/All/tried"

# with open("Python/311devs+peewee/collected_traces.json") as f:
#     pydata = json.load(f)


projects_list = os.listdir(projects_path)
print(len(projects_list))

544


In [76]:
pydata = {}

for project in projects_list[500:545]:
    print(f"Processing project: {project}")
    project_path = os.path.join(projects_path, project)
    trace_file_path = os.path.join(project_path, "collected_traces.json")
    
    if os.path.isfile(trace_file_path):
        try:
            with open(trace_file_path) as f:
                project_data = json.load(f)
                processed_count += 1
            pydata.update(project_data)
        except json.JSONDecodeError as e:
            print(f"Skipping {project}: {e}")

print(f"Processed {processed_count} projects.")

Processing project: keystonetowersystems+siquant
Processing project: pytransitions+transitions
Processing project: arsenetar+dupeguru
Processing project: binux+pyspider
Processing project: facebookresearch+fairseq
Processing project: Hasenpfote+fpq
Processing project: pyinvoke+invoke
Processing project: dschick+udkm1Dsim
Processing project: rkokkelk+python-athom-api
Processing project: dtkav+specific
Processing project: jpadilla+pyjwt
Processing project: twisted+tubes
Processing project: derv82+wifite2
Processing project: mbr+ragstoriches
Processing project: dipu-bd+lightnovel-crawler
Processing project: phelimb+cbg
Processing project: wkschwartz+wrapitup
Processing project: Omega-Cube+graphite-query
Processing project: pyiron+pyiron_base
Processing project: EntilZha+ScalaFunctional
Processing project: getsentry+responses
Processing project: urban48+debpackager
Processing project: al-fontes-jr+bardolph
Processing project: tgbot-collection+YYeTsBot
Processing project: reiinakano+xcessiv

In [81]:
data = []
for k, v in pydata.items():
    code_files = {}
    code_funcs = {}
    fpath = v[0]
    if "/local/rcs/XXX/code/pytrace-collector/logs/pypibugs/tried/" in fpath:
        fpath = fpath.replace(
            "/local/rcs/XXX/code/pytrace-collector/logs/pypibugs/tried/", 
            "/home/XXX/Traces/Python/All/tried/"
        )
    elif "/local/rcs/XXX/code/pytrace-collector/logs/self_collected/tried/" in fpath:
        fpath = fpath.replace(
            "/local/rcs/XXX/code/pytrace-collector/logs/self_collected/tried/", 
            "/home/XXX/Traces/Python/All/tried/"
        )

    
    try:
        with open(fpath, encoding='utf-8') as f:
            code_files[fpath] = f.read()
    except (FileNotFoundError, UnicodeDecodeError) as e:
        print(f"Error reading file {fpath}: {e}")
        try:
            with open(fpath, encoding='unicode_escape') as f:
                code_files[fpath] = f.read()
        except Exception as e2:
            print(f"Error reading file with fallback encoding {fpath}: {e2}")
            continue  # Skip to the next file if reading fails again
    
    try:
        with open(fpath, "rb") as f:
            def get_function_code():
                tree = parser.parse(f.read())
                queue = [tree.root_node]
                while queue:
                    node = queue.pop()
                    if node.type == 'function_definition':
                        function_name = node.child_by_field_name("name").text.decode()
                        function_code = node.text.decode()
                        yield function_name, function_code, node.start_point[0]
                    queue.extend(node.children)
            for func, func_code, func_line in get_function_code():
                code_funcs[func] = (func_code, func_line)
    except FileNotFoundError:
        print(f"File not found: {fpath}")
        continue  # Skip to the next file if the current one is not found
    
    UNKNOWN = v[1]
    pysteps = []
    for step in v[2:]:
        (
            event, # 0
            timestamp, # 1
            line_no, # 2
            source_line, # 3
            new_vars, # 4
            modified_vars, # 5
            ended_by_exception, # 6
            return_value_repr, # 7
            exception, # 8
        ) = step
        if event == "call":
            m = re.match(r"\s*def\s+([^()\s]+)\s*\(([^)]*)\)", source_line)
            func_name = None
            func_code = None
            func_line = None
            if m:
                func_name = m.group(1)
                func_code, func_line = code_funcs.get(func_name, (None, None))

            if len(pysteps) > 0:
                data.append({
                    "function_name": func_name,
                    "function_code": func_code,
                    "function_line": func_line,
                    "fpath": str(fpath),
                    "steps": pysteps,
                    "code": code_files,
                })
            pysteps = []
        pysteps.append({
            "event": event,
            "timestamp": timestamp,
            "line_no": line_no,
            "source_line": source_line,
            "new_vars": new_vars,
            "modified_vars": modified_vars,
            "ended_by_exception": ended_by_exception,
            "return_value_repr": return_value_repr,
            "exception": exception,
        })

    # Ensure we don't append empty steps
    if pysteps:
        data.append({
            "fpath": str(fpath),
            "steps": pysteps,
            "code": code_files,
        })

# Check that all data entries have the 'steps' key
data = [d for d in data if 'steps' in d]

# Create the DataFrame
pydf = pd.DataFrame(data)
pydf["language"] = "Python"
print(pydf.columns, len(pydf))
print(pydf["steps"].str.len().describe())
print(pydf["fpath"].iloc[0])
print(pydf["function_name"].notna().mean(), pydf["function_code"].notna().mean())

Index(['fpath', 'steps', 'code', 'function_name', 'function_code',
       'function_line', 'language'],
      dtype='object') 687614
count    687614.000000
mean          5.382366
std          21.857607
min           1.000000
25%           3.000000
50%           3.000000
75%           3.000000
max        5009.000000
Name: steps, dtype: float64
/home/XXX/Traces/Python/All/tried/pytransitions+transitions/pytransitions+transitions/tests/test_add_remove.py
0.12366531222459112 0.12366531222459112


In [82]:
# Format the Python traces into df.
# Eventually we will have traces from all languages.

df = pydf.copy()
df = df[df["function_name"].notna() & df["function_code"].notna() & df["function_line"].notna()]

def format_steps(steps):
    if not steps[0]["source_line"].lstrip().startswith("def"):
        return None

    function_input = ", ".join([f"{v[0]}" for k, v in steps[0]["new_vars"].items()])
    function_output = steps[-1]["return_value_repr"]
    
    
    trace_steps = []
    for i, step in enumerate(steps[:-1]):
        # TODO: Handle exception, jump, multiple assignments within a line, etc.
        # TODO: Handle duplicate lines
        if step["event"] == "exception":
            trace_steps.append({
                "line_no": step["line_no"],
                "values_updated": [{"name": k, "type": v[1], "value": v[0]} for k, v in step["new_vars"].items()],
                "exception": step["exception"]  
            })
            continue
            
        
        #values_updated = steps[i+1]["new_vars"]
        updated_vars = {**steps[i+1].get("new_vars", {}), **steps[i+1].get("modified_vars", {})}
        values_updated = [{"name": k, "type": v[1], "value": v[0]} for k, v in updated_vars.items()]
        trace_steps.append({"line_no": step["line_no"], "values_updated": values_updated})
    return trace_steps, function_input, function_output


# Use `apply` and `zip` to unpack the results from format_steps
df[["trace_steps", "function_input", "function_output"]] = df["steps"].apply(lambda steps: pd.Series(format_steps(steps)))

# Select relevant columns for final DataFrame
df = df[["function_name", "function_code", "function_line", "trace_steps", "function_input", "function_output"]]

# df = df.drop(columns=["steps"])
# format_steps(df["steps"].iloc[100])
# df = df.rename(columns={"function_name": })
print(len(df), df.columns)
print(df["trace_steps"].iloc[0][0].keys())

85034 Index(['function_name', 'function_code', 'function_line', 'trace_steps',
       'function_input', 'function_output'],
      dtype='object')
dict_keys(['line_no', 'values_updated'])


In [83]:
def is_simple_data(value):
    """Check if the value is a simple data type."""
    if isinstance(value, (int, float, str, bool)):
        return True
    elif isinstance(value, (list, tuple, set)):
        return all(is_simple_data(v) for v in value)
    elif isinstance(value, dict):
        return all(is_simple_data(k) and is_simple_data(v) for k, v in value.items())
    elif isinstance(value, type(None)):
        return True
    return False

def contains_complex_type(value):
    """Check if a string contains complex types."""
    complex_patterns = [
        r"<class '.*'>",               # e.g., "<class 'peewee.SqliteDatabase'>"
        r"<.*object at 0x[\da-f]+>",    # e.g., "<peewee.PrimaryKeyField object at 0x7f9ad1c4d550>"
        r"at 0x[\da-f]+",               # any memory address
        r"^\s*<",                       # any string starting with "<"
    ]
    
    for pattern in complex_patterns:
        if re.search(pattern, value):
            return True
    return False


def filter_complex_entries(data_entry):
    """Filter out entries with complex types in the input or output."""
    try:
        parsed_input = ast.literal_eval(tup.function_input)
        parsed_output = ast.literal_eval(tup.function_output)
    except (ValueError, SyntaxError):
        # If parsing fails, assume it's due to complex types
        return False

    # Check for complex types in parsed input/output
    if not is_simple_data(parsed_input) or contains_complex_type(tup.function_input):
        return False
    
    if not is_simple_data(parsed_output) or contains_complex_type(tup.function_output):
        return False
    
    return True


# Function to calculate cyclomatic complexity
def calculate_complexity(code):
    tree = ast.parse(code)
    visitor = PathGraphingAstVisitor()
    visitor.preorder(tree, visitor)
    complexity = list(visitor.graphs.values())[0].complexity()
    return complexity

def format_code(code):
    # Remove blank lines
    code = "".join(line for line in code.splitlines(keepends=True) if line)
    return code


def source_code_formatting(function_code, function_name, function_input):
    lines = function_code.splitlines(keepends=False)
    return "\n".join(lines)

In [80]:
import json
import ast
from mccabe import PathGraphingAstVisitor
from collections import defaultdict

COMMENT_SYMBOL = "#"

source_code_map = defaultdict(list)
jsonl_data = []

for tup in df.itertuples():
    if not isinstance(tup.trace_steps, list):
        continue
    
    if not filter_complex_entries(tup):
        continue
    
    if tup.function_input is None or any(x in str(tup.function_input).lower() for x in ["none"]):
        continue
    
    if ("tensor" in tup.function_input or "Sequential" in tup.function_input or "array" in tup.function_input or
                "ReLU()" in tup.function_input or "Encoder()" in tup.function_input) and len(tup.function_input) > 30:
        continue
    
    values_per_line = defaultdict(list)
    repeated_values = defaultdict(list)
    step_index = 1
    
    for step in tup.trace_steps:
        if "exception" in step:
            exception_message = f'EXCEPTION: {step["exception"]}'
            values_per_line[step["line_no"]-tup.function_line].append((step_index, exception_message))
            step_index +=1
        else:
            for value in step["values_updated"]:
                state = f'{value["name"]} = {str(value["value"])}'
                line_position = step["line_no"] - tup.function_line

                if state not in repeated_values[line_position]:
                    repeated_values[line_position].append(state)

            if len(repeated_values[step["line_no"]-tup.function_line]) > 0:
                state_values = repeated_values[step["line_no"]-tup.function_line]
                values_per_line[step["line_no"]-tup.function_line].append((step_index, state_values))
                step_index += 1
    
    if len(values_per_line) == 0:
        continue
    
    source_code_with_input = source_code_formatting(tup.function_code, tup.function_name, tup.function_input)

    lines = tup.function_code.splitlines(keepends=False)
    annotated_lines = []

    function_def_line = lines[0]
    input_variables = f"{COMMENT_SYMBOL} <INPUT> {tup.function_input} </INPUT>"
    annotated_lines.append(input_variables+"\n"+function_def_line)

    for lineno, line in enumerate(lines[1:], start=2):
        states_and_line_values = values_per_line[lineno]
        line = line.rstrip()

        if len(states_and_line_values) > 0 and len(states_and_line_values) == 1:
            for i, (index, values) in enumerate(states_and_line_values):
                for state_index, value in enumerate(values, start=1):
                    line += f" {COMMENT_SYMBOL} [STATE] " + "".join(value) + " [/STATE]"
        annotated_lines.append(line)
        
    output_variables = f"{COMMENT_SYMBOL} <OUTPUT> {tup.function_output} </OUTPUT>"
    annotated_lines.append(output_variables)

    function_call_with_input = f"{tup.function_name}({tup.function_input})"
    annotated_lines.append("\n" + function_call_with_input)

    annotated_function_code = "\n".join(annotated_lines)
    formatted_code = format_code(annotated_function_code)

    existing_inputs = {entry['input'] for entry in source_code_map[tup.function_code]}
    if tup.function_input not in existing_inputs:
        source_code_map[tup.function_code].append({
            "input": tup.function_input,
            "output": tup.function_output,
            "scratchpad_format": formatted_code
        })

for source_code, entries in source_code_map.items():
    if len(entries) == 1:
        jsonl_data.append({
            "Source Code": source_code,
            "input": entries[0]["input"],
            "output": entries[0]["output"],
            "scratchpad_format": entries[0]["scratchpad_format"]
        })
    else:
        jsonl_data.append({
            "Source Code": source_code,
            "input": [entry["input"] for entry in entries],
            "output": [entry["output"] for entry in entries],
            "scratchpad_format": [entry["scratchpad_format"] for entry in entries]
        })

with open("Python_Dataset.jsonl", "a", encoding='utf-8') as file:
    for entry in jsonl_data:
        file.write(json.dumps(entry) + "\n")

In [84]:
import json


with open("Python_Dataset.jsonl", "r", encoding='utf-8') as file:
    data = [json.loads(line) for line in file]


multi_entries = [entry for entry in data if isinstance(entry.get("input"), list)]


for i, entry in enumerate(multi_entries[:5]):  
    print(f"\nEntry {i+1}:")
    print(f"Source Code:\n{entry['Source Code']}\n")
    
    print("Inputs and their corresponding scratchpad formats:")
    for input_val, scratchpad in zip(entry['input'], entry['scratchpad_format']):
        print(f"\nInput: {input_val}")
        print(f"Scratchpad format (first 100 chars): {scratchpad}...")
    
    print("\n" + "="*80 + "\n")

print(f"Total entries with multiple variations: {len(multi_entries)}")


Entry 1:
Source Code:
def exchange(a, i, j):
    temp = a[i]
    a[i] = a[j]
    a[j] = temp

Inputs and their corresponding scratchpad formats:

Input: [0, 100, 200, 0, 0, 0, 0, 0, 0, 0], 2, 1
Scratchpad format (first 100 chars): # <INPUT> [0, 100, 200, 0, 0, 0, 0, 0, 0, 0], 2, 1 </INPUT>
def exchange(a, i, j):
    temp = a[i] # [STATE] temp = 200 [/STATE]
    a[i] = a[j] # [STATE] a = [0, 100, 100, 0, 0, 0, 0, 0, 0, 0] [/STATE]
    a[j] = temp # [STATE] a = [0, 200, 100, 0, 0, 0, 0, 0, 0, 0] [/STATE]
# <OUTPUT> None </OUTPUT>

exchange([0, 100, 200, 0, 0, 0, 0, 0, 0, 0], 2, 1)...

Input: [0, 200, 100, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1, 22
Scratchpad format (first 100 chars): # <INPUT> [0, 200, 100, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1, 22 </INPUT>
def exchange(a, i, j):
    temp = a[i] # [STATE] temp = 200 [/

# Making the Dataset with ScratchPad Formatting

In [34]:
import random
import re

def process_code(code):
    lines = code.split("\n")
    state_lines = []

    for idx, line in enumerate(lines):
        if "# [STATE]" in line:
            state_lines.append(idx)


    if state_lines:
        selected_idx = random.choice(state_lines)
        selected_line = lines[selected_idx]
        # ground_truth_line = selected_line.split("# [STATE]")[1].strip()

        match = re.search(r'\[STATE\](.*?)\[/STATE\]', selected_line)
        if match:
            ground_truth_line = match.group(1)
        else:
            ground_truth_line = None


        # indentation = re.match(r"^\s*", selected_line).group()
        # Modify the selected line
        lines[selected_idx] = selected_line.split("# [STATE]")[0]

        # Safely split the ground truth line on the first "="
        if ground_truth_line:
            key_value_split = ground_truth_line.split("=", 1)
            if len(key_value_split) == 2:
                key, value = key_value_split[0].strip(), key_value_split[1].strip()

                indentation = re.match(r"^\s*", selected_line).group()
                lines.insert(selected_idx + 1, f"{indentation}assert {key} == ??")

                assertion_line = f"assert {key} == {value}"

                # Remove all comments from subsequent lines
                for i in range(selected_idx + 2, len(lines)):
                    lines[i] = lines[i].split("# [STATE]")[0]

            else:
                # Handle the case where the line doesn't have an "=" sign or doesn't follow the expected format
                return None, None, None
        else:
            # Handle the case where there is no ground truth
            return None, None, None

    # If no state lines, process based on rule (ii)
    else:
        random_idx = random.randint(2, len(lines) - 4)
        selected_line = lines[random_idx]

        indentation = re.match(r"^\s*", selected_line).group()
        lines[random_idx] = selected_line
        lines.insert(random_idx + 1, f"{indentation}assert {selected_line.split('=')[0].strip()} == ??")

        assertion_line = f"assert {selected_line.split('=')[0].strip()} == None"
        value = "None"

        # Remove all comments from subsequent lines
        for i in range(random_idx + 2, len(lines)):
            lines[i] = lines[i].split("# <STATE>")[0].strip()

    #print(lines[-3])
    # Join the modified code
    modified_code = "\n".join([line for i, line in enumerate(lines) if line.strip() and i != len(lines) - 3])

    return modified_code, assertion_line, value


# Example Usage
input_code = "# <INPUT> 0, 0, 86400 </INPUT>\ndef _input(x):\n        return name_lookup(x) or func_lookup(x) or default\n# <OUTPUT> 0.0 </OUTPUT>\n\n_input(0, 0, 86400)"

formatted_code, assertion_line, value = process_code(input_code)
print("Formatted Code:")
print(formatted_code)
print("\nAssertion Line:")
print(assertion_line)
print("Value")
print(value)


Formatted Code:
# <INPUT> 0, 0, 86400 </INPUT>
def _input(x):
        return name_lookup(x) or func_lookup(x) or default
        assert return name_lookup(x) or func_lookup(x) or default == ??
_input(0, 0, 86400)

Assertion Line:
assert return name_lookup(x) or func_lookup(x) or default == None
Value
None


In [35]:
import json

source_code_set = set()

def process_jsonl(input_file, output_file):
    # Open the input and output files
    idx = 0
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            # Parse each line as JSON
            data = json.loads(line)
            
            # Extract the "scratchpad_format" key
            source_code = data.get('Source Code')
            code = data.get("scratchpad_format")
            if not code or source_code in source_code_set:
                continue  # Skip lines without "scratchpad_format"
            
            # Process the code
            formatted_code, assertion_line, value = process_code(code)
            
            if formatted_code is not None:
            # Create the new JSON object
                new_data = {
                    "idx" : idx,
                    "Code": formatted_code,
                    "Assertion Line": assertion_line,
                    "Assertion Value": value
                }
                source_code_set.add(source_code)
                idx += 1
                
                # Write the new JSON object to the output file
                outfile.write(json.dumps(new_data) + "\n")


# Specify file names
input_jsonl = "fine_tune_data.jsonl"
output_jsonl = "intermediate_value_dataset_1.jsonl"

# Run the processing
process_jsonl(input_jsonl, output_jsonl)
print(f"Processing complete. Saved to {output_jsonl}")


Processing complete. Saved to intermediate_value_dataset_1.jsonl


# Processing Intermediate Value dataset (Discard statements after assert)

In [21]:
import json


def process_code(code, assertion_line):
    lines = code.split('\n')
    

    assertion_index = None
    
    for i, line in enumerate(lines):
        if "assert" in line.strip():
            if line.strip().find(assertion_line):
                assertion_index = i
                break         
        
        # if "assert" in line.strip() and assertion_line in line.split("==")[0].strip():
        #     assertion_index = i
        #     break

    if assertion_index is not None:
        processed_lines = lines[:assertion_index + 1]
    else:
        processed_lines = lines  



    return '\n'.join(processed_lines)


input_file = "intermediate_value_dataset_1.jsonl"
output_file = "intermediate_value_dataset_dis.jsonl"


with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:

        data = json.loads(line)
        

        processed_code = process_code(data["Code"], data["Assertion Line"])
        

        data["Code"] = processed_code

        outfile.write(json.dumps(data) + "\n")
        
        print(processed_code)
        print("\n" + "="*80 + "\n")

print(f"Processed data saved to {output_file}")


# <INPUT> 'testing', '/home/XXX/.tsecrets' </INPUT>
def secrets_dir(env=os.getenv('D2_ENVIRONMENT', None),
                basedir=os.getenv('D2_SECRETS_BASEDIR', None)):
    if env is not None:
        env_str = str(env) 
        assert env_str == ??


# <INPUT> 'testing' </INPUT>
def _identify_environment(environment=None):
    """
    Returns the environment identifier.
    There are multiple ways to define the default environment (in order
    of priority):
    1. The --environment command line option;
    2. The content of the file .python_secrets_environment in the current
       working directory;
    3. The value specified by environment variable D2_ENVIRONMENT; or
    4. The basename of the current working directory.
    """
    cwd = os.getcwd() 
    assert cwd == ??


# <INPUT> [{'Variable': 'jenkins_admin_password', 'Type': 'password'}, {'Variable': 'ca_rootca_password', 'Type': 'password'}], 'Variable', 'something_not_there' </INPUT>
def find(lst, key, value):
    for i, d